# Bathymetry Contour Visualization - Texas-Louisiana Shelf

This notebook creates detailed bathymetry contour maps for the Texas-Louisiana shelf region using GEBCO 2024 sub-ice topography data. The visualization includes depth contour lines, filled bathymetry, and geographic features.

**Data Source:**
- Bathymetry data: `E:\geodata\gebco_2024_sub_ice_topo\GEBCO_2024_sub_ice_topo.nc`
- GEBCO 2024 sub-ice topography dataset

**Study Region:**
- Longitude: -95.0° to -88.0°W  
- Latitude: 27.5° to 30.5°N
- Focus: Texas-Louisiana continental shelf

**Features:**
- Filled bathymetry visualization
- Depth contour lines with labels
- Coastline and geographic features
- Customizable contour levels and styling

### Import Required Libraries

In [ ]:
# Import essential libraries
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import matplotlib.colors as mcolors
import cmocean

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

print("Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Xarray version: {xr.__version__}")

### Configuration Parameters

In [ ]:
# --- Geographical Bounds ---
lon_min, lon_max = -95.0, -88.0  # Longitude range (degrees West)
lat_min, lat_max = 27.5, 30.5    # Latitude range (degrees North)

# --- Data File Path ---
bathymetry_file = r"E:\geodata\gebco_2024_sub_ice_topo\GEBCO_2024_sub_ice_topo.nc"

# --- Contour Level Configuration ---
# Depth contours in meters (negative values for below sea level)
contour_levels = [-200, -100, -75, -50, -40, -30, -20, -15, -10, -5]
major_contour_levels = [-200, -100, -50, -20, -10]  # Emphasized contours

# --- Styling Parameters ---
contour_colors = 'black'
contour_linewidths = 0.8
major_contour_linewidths = 1.2
contour_alpha = 0.8
filled_alpha = 0.7

# --- Colormap Configuration ---
bathymetry_cmap = cmocean.cm.deep_r  # Reversed deep colormap for bathymetry
depth_range = (-250, 10)  # Color scale range in meters

print(f"Study Region: Lon ({lon_min}, {lon_max}), Lat ({lat_min}, {lat_max})")
print(f"Bathymetry file: {bathymetry_file}")
print(f"Contour levels: {contour_levels}")
print(f"Major contour levels: {major_contour_levels}")
print(f"Depth color range: {depth_range} meters")

### Load Bathymetry Data

In [ ]:
# Load GEBCO bathymetry data
print("Loading GEBCO bathymetry data...")
try:
    # Open the NetCDF dataset
    bathy_ds = xr.open_dataset(bathymetry_file)
    print(f"Dataset loaded successfully")
    print(f"Dataset variables: {list(bathy_ds.data_vars)}")
    print(f"Dataset coordinates: {list(bathy_ds.coords)}")
    
    # Subset bathymetry to study region
    print("\nSubsetting data to study region...")
    bathy_subset = bathy_ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    
    # Extract bathymetry data and coordinates
    bathy_data = bathy_subset['elevation']  # GEBCO uses 'elevation' as variable name
    bathy_lon = bathy_subset['lon']
    bathy_lat = bathy_subset['lat']
    
    # Display data information
    print(f"\nBathymetry data successfully loaded:")
    print(f"  Data shape: {bathy_data.shape}")
    print(f"  Longitude range: {float(bathy_lon.min()):.3f} to {float(bathy_lon.max()):.3f}°")
    print(f"  Latitude range: {float(bathy_lat.min()):.3f} to {float(bathy_lat.max()):.3f}°")
    print(f"  Elevation range: {float(bathy_data.min()):.1f} to {float(bathy_data.max()):.1f} meters")
    print(f"  Resolution: ~{float(bathy_lon[1] - bathy_lon[0]):.4f}° longitude")
    print(f"              ~{float(bathy_lat[1] - bathy_lat[0]):.4f}° latitude")
    
    # Calculate some basic statistics
    land_mask = bathy_data > 0
    ocean_mask = bathy_data <= 0
    
    print(f"\nData Statistics:")
    print(f"  Land pixels: {int(land_mask.sum())} ({100*float(land_mask.sum())/bathy_data.size:.1f}%)")
    print(f"  Ocean pixels: {int(ocean_mask.sum())} ({100*float(ocean_mask.sum())/bathy_data.size:.1f}%)")
    if ocean_mask.sum() > 0:
        ocean_depths = bathy_data.where(ocean_mask)
        print(f"  Ocean depth range: {float(ocean_depths.min()):.1f} to {float(ocean_depths.max()):.1f} meters")
        print(f"  Mean ocean depth: {float(ocean_depths.mean()):.1f} meters")
    
    bathy_available = True
    
except Exception as e:
    print(f"Error loading bathymetry data: {e}")
    print("Cannot proceed without bathymetry data.")
    bathy_available = False

### Create Basic Bathymetry Map

In [ ]:
if bathy_available:
    # Create a basic filled bathymetry map
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Set map extent
    extent = [lon_min, lon_max, lat_min, lat_max]
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Create filled bathymetry plot
    bathy_plot = ax.pcolormesh(bathy_lon, bathy_lat, bathy_data,
                              cmap=bathymetry_cmap, shading='auto',
                              vmin=depth_range[0], vmax=depth_range[1],
                              alpha=filled_alpha, transform=ccrs.PlateCarree())
    
    # Add geographic features
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black', zorder=3)
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.8, zorder=2)
    ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', zorder=2)
    ax.add_feature(cfeature.STATES, linestyle='-', edgecolor='gray', linewidth=0.5, zorder=2)
    
    # Add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, 
                      color='gray', alpha=0.7, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    # Add colorbar
    cbar = plt.colorbar(bathy_plot, ax=ax, orientation='vertical', 
                       shrink=0.8, pad=0.05, aspect=30)
    cbar.set_label('Elevation (meters)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)
    
    # Set title
    plt.title('Bathymetry of the Texas-Louisiana Shelf\nGEBCO 2024 Data', 
              fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.show()
    
    print("Basic bathymetry map created successfully")
else:
    print("Cannot create map: bathymetry data not available")

### Add Contour Lines and Labels

In [ ]:
if bathy_available:
    # Create bathymetry map with contour lines
    fig = plt.figure(figsize=(14, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Set map extent
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Create filled bathymetry plot
    bathy_plot = ax.pcolormesh(bathy_lon, bathy_lat, bathy_data,
                              cmap=bathymetry_cmap, shading='auto',
                              vmin=depth_range[0], vmax=depth_range[1],
                              alpha=filled_alpha, transform=ccrs.PlateCarree())
    
    # Add contour lines for all levels
    contours = ax.contour(bathy_lon, bathy_lat, bathy_data, 
                         levels=contour_levels, colors=contour_colors, 
                         linewidths=contour_linewidths, alpha=contour_alpha,
                         transform=ccrs.PlateCarree())
    
    # Add contour labels
    labels = ax.clabel(contours, inline=True, fontsize=9, fmt='%d', 
                      inline_spacing=8, manual=False)
    
    # Style the contour labels
    for label in labels:
        label.set_fontweight('bold')
        label.set_bbox(dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))
    
    # Add geographic features
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black', zorder=4)
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.9, zorder=3)
    ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', zorder=3)
    ax.add_feature(cfeature.STATES, linestyle='-', edgecolor='gray', linewidth=0.5, zorder=3)
    
    # Add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, 
                      color='gray', alpha=0.7, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    # Add colorbar
    cbar = plt.colorbar(bathy_plot, ax=ax, orientation='vertical', 
                       shrink=0.8, pad=0.05, aspect=30)
    cbar.set_label('Elevation (meters)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)
    
    # Set title and subtitle
    plt.title('Bathymetry with Depth Contours\nTexas-Louisiana Shelf (GEBCO 2024)', 
              fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Bathymetry map with {len(contour_levels)} contour levels created")
    print(f"Contour levels: {contour_levels} meters")
else:
    print("Cannot create contour map: bathymetry data not available")

### Customize Contour Styling

In [ ]:
if bathy_available:
    # Create bathymetry map with customized contour styling
    fig = plt.figure(figsize=(14, 9))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Set map extent
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Create filled bathymetry plot
    bathy_plot = ax.pcolormesh(bathy_lon, bathy_lat, bathy_data,
                              cmap=bathymetry_cmap, shading='auto',
                              vmin=depth_range[0], vmax=depth_range[1],
                              alpha=filled_alpha, transform=ccrs.PlateCarree())
    
    # Add regular contour lines
    regular_levels = [level for level in contour_levels if level not in major_contour_levels]
    if regular_levels:
        contours_regular = ax.contour(bathy_lon, bathy_lat, bathy_data, 
                                     levels=regular_levels, colors='darkblue', 
                                     linewidths=contour_linewidths, alpha=contour_alpha,
                                     transform=ccrs.PlateCarree())
    
    # Add major contour lines (emphasized)
    contours_major = ax.contour(bathy_lon, bathy_lat, bathy_data, 
                               levels=major_contour_levels, colors='navy', 
                               linewidths=major_contour_linewidths, alpha=1.0,
                               transform=ccrs.PlateCarree())
    
    # Add labels for major contours only
    labels_major = ax.clabel(contours_major, inline=True, fontsize=10, fmt='%d m', 
                            inline_spacing=10, manual=False)
    
    # Style the major contour labels
    for label in labels_major:
        label.set_fontweight('bold')
        label.set_color('navy')
        label.set_bbox(dict(boxstyle='round,pad=0.3', facecolor='white', 
                           edgecolor='navy', alpha=0.9, linewidth=0.5))
    
    # Add geographic features
    ax.add_feature(cfeature.COASTLINE, linewidth=1.2, edgecolor='black', zorder=5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.9, zorder=4)
    ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', zorder=4)
    ax.add_feature(cfeature.STATES, linestyle='-', edgecolor='gray', linewidth=0.6, zorder=4)
    
    # Add gridlines with better formatting
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, 
                      color='gray', alpha=0.6, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 10}
    gl.ylabel_style = {'size': 10}
    
    # Add colorbar with better positioning
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
    cbar = plt.colorbar(bathy_plot, cax=cbar_ax, orientation='vertical')
    cbar.set_label('Elevation (meters)', fontsize=12, labelpad=15)
    cbar.ax.tick_params(labelsize=10)
    
    # Add title with more detailed information
    plt.suptitle('Bathymetry of the Texas-Louisiana Continental Shelf', 
                fontsize=16, fontweight='bold', y=0.95)
    ax.set_title(f'Depth Contours: {major_contour_levels} meters (major), GEBCO 2024 Data', 
                fontsize=12, pad=15)
    
    # Create legend for contour types
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='navy', linewidth=major_contour_linewidths, 
               label=f'Major contours ({major_contour_levels} m)'),
        Line2D([0], [0], color='darkblue', linewidth=contour_linewidths, 
               label='Minor contours')
    ]
    ax.legend(handles=legend_elements, loc='lower left', 
             bbox_to_anchor=(0.02, 0.02), frameon=True, fancybox=True, shadow=True)
    
    plt.subplots_adjust(right=0.9)
    plt.show()
    
    print("Customized contour map created successfully")
    print(f"Major contours: {major_contour_levels} meters")
    print(f"Minor contours: {regular_levels} meters")
else:
    print("Cannot create customized map: bathymetry data not available")

### Create Multi-Level Contour Visualization

In [ ]:
if bathy_available:
    # Create a comprehensive multi-level visualization
    fig, axes = plt.subplots(1, 2, figsize=(20, 8), 
                            subplot_kw={'projection': ccrs.PlateCarree()})
    
    # === LEFT PANEL: Filled Contours ===
    ax1 = axes[0]
    ax1.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Create filled contour plot
    filled_contours = ax1.contourf(bathy_lon, bathy_lat, bathy_data,
                                  levels=np.arange(depth_range[0], depth_range[1]+5, 10),
                                  cmap=bathymetry_cmap, alpha=0.8,
                                  transform=ccrs.PlateCarree(), extend='both')
    
    # Add line contours on top
    line_contours1 = ax1.contour(bathy_lon, bathy_lat, bathy_data, 
                                levels=contour_levels, colors='black', 
                                linewidths=0.6, alpha=0.8,
                                transform=ccrs.PlateCarree())
    
    # Add geographic features
    ax1.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black', zorder=3)
    ax1.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.9, zorder=2)
    ax1.add_feature(cfeature.STATES, linestyle='-', edgecolor='gray', linewidth=0.5, zorder=2)
    
    # Add gridlines
    gl1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, 
                       color='gray', alpha=0.6, linestyle='--')
    gl1.top_labels = False
    gl1.right_labels = False
    
    ax1.set_title('Filled Bathymetry Contours', fontsize=12, pad=15)
    
    # === RIGHT PANEL: Line Contours with Labels ===
    ax2 = axes[1]
    ax2.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Create basic bathymetry background
    bathy_bg = ax2.pcolormesh(bathy_lon, bathy_lat, bathy_data,
                             cmap='Greys', shading='auto',
                             vmin=depth_range[0], vmax=depth_range[1],
                             alpha=0.3, transform=ccrs.PlateCarree())
    
    # Add detailed contour lines
    line_contours2 = ax2.contour(bathy_lon, bathy_lat, bathy_data, 
                                levels=contour_levels, colors='darkblue', 
                                linewidths=1.0, alpha=1.0,
                                transform=ccrs.PlateCarree())
    
    # Add contour labels
    labels2 = ax2.clabel(line_contours2, inline=True, fontsize=9, fmt='%d', 
                        inline_spacing=12, manual=False)
    
    # Style the labels
    for label in labels2:
        label.set_fontweight('bold')
        label.set_color('darkblue')
        label.set_bbox(dict(boxstyle='round,pad=0.2', facecolor='white', 
                           edgecolor='darkblue', alpha=0.9, linewidth=0.5))
    
    # Add geographic features
    ax2.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black', zorder=3)
    ax2.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.9, zorder=2)
    ax2.add_feature(cfeature.STATES, linestyle='-', edgecolor='gray', linewidth=0.5, zorder=2)
    
    # Add gridlines
    gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, 
                       color='gray', alpha=0.6, linestyle='--')
    gl2.top_labels = False
    gl2.right_labels = False
    
    ax2.set_title('Labeled Depth Contours', fontsize=12, pad=15)
    
    # Add shared colorbar for filled contours
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
    cbar = plt.colorbar(filled_contours, cax=cbar_ax, orientation='vertical')
    cbar.set_label('Elevation (meters)', fontsize=12, labelpad=15)
    cbar.ax.tick_params(labelsize=10)
    
    # Add overall title
    plt.suptitle('Texas-Louisiana Shelf Bathymetry: Multi-Level Contour Analysis\nGEBCO 2024 Data', 
                fontsize=16, fontweight='bold', y=0.95)
    
    plt.subplots_adjust(top=0.85, right=0.9, wspace=0.15)
    plt.show()
    
    # Print summary information
    print("Multi-level contour visualization created successfully")
    print("\n=== Bathymetry Analysis Summary ===")
    print(f"Study area: {lon_min}°W to {lon_max}°W, {lat_min}°N to {lat_max}°N")
    print(f"Data resolution: ~{float(bathy_lon[1] - bathy_lon[0]):.4f}° (~{float(bathy_lon[1] - bathy_lon[0])*111:.1f} km)")
    print(f"Depth range: {float(bathy_data.min()):.1f} to {float(bathy_data.max()):.1f} meters")
    print(f"Contour levels: {len(contour_levels)} levels from {min(contour_levels)} to {max(contour_levels)} meters")
    
    # Calculate depth statistics for different zones
    shallow_mask = (bathy_data <= 0) & (bathy_data > -20)
    mid_mask = (bathy_data <= -20) & (bathy_data > -100)
    deep_mask = bathy_data <= -100
    
    print(f"\n=== Depth Zone Distribution ===")
    if shallow_mask.sum() > 0:
        print(f"Shallow water (0 to -20m): {100*float(shallow_mask.sum())/bathy_data.size:.1f}% of area")
    if mid_mask.sum() > 0:
        print(f"Mid-depth (-20 to -100m): {100*float(mid_mask.sum())/bathy_data.size:.1f}% of area")
    if deep_mask.sum() > 0:
        print(f"Deep water (>-100m): {100*float(deep_mask.sum())/bathy_data.size:.1f}% of area")
    
else:
    print("Cannot create multi-level visualization: bathymetry data not available")

### Summary and Notes

This notebook provides comprehensive bathymetry visualization tools for the Texas-Louisiana shelf region using GEBCO 2024 data. The visualizations include:

**Key Features:**
1. **Basic Bathymetry Map**: Color-filled elevation with geographic features
2. **Contour Lines**: Depth contours with labeled depths
3. **Customized Styling**: Major/minor contour differentiation
4. **Multi-level Visualization**: Side-by-side filled and line contours

**Technical Details:**
- **Data Source**: GEBCO 2024 sub-ice topography dataset
- **Coordinate System**: WGS84 Geographic (PlateCarree)
- **Contour Levels**: Customizable depth intervals in meters
- **Color Schemes**: Ocean-appropriate colormaps from cmocean

**Customization Options:**
- Modify `contour_levels` for different depth intervals
- Adjust `major_contour_levels` to emphasize specific depths
- Change geographic bounds by modifying `lon_min/max` and `lat_min/max`
- Customize colors, line widths, and transparency settings

**Applications:**
- Oceanographic research and analysis
- Coastal and marine spatial planning
- Navigation and maritime safety
- Habitat mapping and environmental studies
- Background for other oceanographic data visualization

The bathymetry contours provide essential context for understanding water depth patterns, submarine topography, and the continental shelf structure in the Gulf of Mexico region.